In [ ]:
# main.py
from initializations import Initializations
from trading_data import TradingData
from order_executor import OrderExecutor, get_token
from post_order_processor import PostOrderProcessor
import datetime
import time
import logging

# グローバルな設定
API_PASSWORD = "1995taka"
ORDER_PASSWORD = "1995tAkA@@"

#インデックス 295~300
# SYMBOL = "1579"
#NTT 156.8
SYMBOL = "9432"

EXCHANGE = 1  # 東証プライム市場を設定
SLEEP_INTERVAL = 0.3 # 待機時間（秒）

def setup_logger():
    """
    ロガーを設定します。
    """
    logger = logging.getLogger('TradingLogger')
    logger.setLevel(logging.INFO)
    
    # 既にハンドラーが設定されている場合は追加しない
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    # ロガーの伝播を無効化
    logger.propagate = False
    return logger

def main():
    # 初期化クラスのインスタンスを作成
    init = Initializations()
    init.order_password = ORDER_PASSWORD
    init.symbol = SYMBOL
    init.exchange = EXCHANGE
    # ロガーの設定
    init.logger = setup_logger()
    # トークンを取得
    token = get_token(API_PASSWORD)
    init.token = token
    # TradingDataクラスのインスタンスを作成
    trading_data = TradingData(init, token)
    # OrderExecutorクラスのインスタンスを作成
    order_executor = OrderExecutor(init, trading_data, token, ORDER_PASSWORD)
    # PostOrderProcessorクラスのインスタンスを作成
    post_order_processor = PostOrderProcessor(init)
    # 初回の価格取得
    initial_price = trading_data.fetch_current_price()
    init.previous_price = initial_price
    init.current_price = initial_price
    init.logger.info(f"初回価格を設定しました。価格: {initial_price}")
    # トレーディングループを開始
    while True:
        # 最新の株価データを取得
        current_price = trading_data.fetch_current_price()
        # price = (current_price)
        if current_price is not None:
            # データが4つになったらOHLCを生成
            if len(init.prices) >= 4:
                trading_data.create_ohlc()
                # Buy and Holdのエクイティカーブを計算
                trading_data.calculate_buy_and_hold_equity()
                # テクニカル指標の計算
                trading_data.calculate_technical_indicators()
                # 最新のテクニカル指標データを取得
                band_width = init.df['band_width']
                hist = init.df['hist']
                di_difference = init.df['di_difference']
                adx_difference = init.df['adx_difference']
                trading_data.update_latest_9_data(band_width, hist, di_difference, adx_difference)
                init.logger.info(f"Interpolated data length: {len(init.interpolated_data)}")
                # トレーディング状態の計算と更新
                post_order_processor.calculate_trading_values(datetime.datetime.now())
                # シグナルを生成
                trading_data.generate_signals(init.interpolated_data, init.R1, init.R2, init.R3, init.S1, init.S2, init.S3)
                # 注文の実行
                order_executor.execute_orders()
                # 補間データとOHLCデータを表示
                # trading_data.display_interpolated_data()
        # 指定した間隔（秒数）待機
        time.sleep(SLEEP_INTERVAL)

if __name__ == '__main__':
    main()